__DS_challenge_1__

In [1]:
!ls /home/dsc/Data/challenge

bookings.csv.bz2  searches.csv.bz2


In [2]:
!bzcat /home/dsc/Data/challenge/bookings.csv.bz2 | head -10000 >> bookings.sample.csv


bzcat: I/O or other error, bailing out.  Possible reason follows.
bzcat: Broken pipe
	Input file = /home/dsc/Data/challenge/bookings.csv.bz2, output file = (stdout)


In [3]:
!bzip2 -f bookings.sample.csv

In [4]:
!ls -l

total 744
-rw-rw-r-- 1 dsc dsc 535893 nov 21 10:20 bookings.sample.csv.bz2
-rw-rw-r-- 1 dsc dsc  79933 nov 20 20:33 Exercise_1_and_2.ipynb
-rw-rw-r-- 1 dsc dsc 138915 nov 21 09:55 Exercise_3.ipynb
-rw-rw-r-- 1 dsc dsc     72 nov 20 17:17 README.md


In [5]:
!bzcat bookings.sample.csv.bz2 | wc -l

10000


With Python

In [6]:
import bz2

fileBz2 = bz2.BZ2File('./bookings.sample.csv.bz2')
type(fileBz2)

bz2.BZ2File

In [7]:
k = 0
for line in fileBz2:
    k += 1
    
print(k)

10000


Or

In [8]:
with bz2.BZ2File('./bookings.sample.csv.bz2') as fileBz2:
    k = 0
    for line in fileBz2:
        k += 1
        
print(k)

10000


This method close the file at the end

In [9]:
fileBz2.closed

True

In [10]:
with bz2.BZ2File('./bookings.sample.csv.bz2') as fileBz2:
    k = 0
    for line in fileBz2:
        k += 1
        if k == 1000:
            break
print(k)

1000


__Exercise 2__

In [11]:
import pandas as pd

In [12]:
bookings_sample = pd.read_csv('./bookings.sample.csv.bz2', compression = 'bz2', sep = '^')
bookings_sample.shape

(9999, 38)

In [16]:
len(bookings_sample['rloc          '].unique())

3638

In [14]:
bookings_sample.columns

Index(['act_date           ', 'source', 'pos_ctry', 'pos_iata', 'pos_oid  ',
       'rloc          ', 'cre_date           ', 'duration', 'distance',
       'dep_port', 'dep_city', 'dep_ctry', 'arr_port', 'arr_city', 'arr_ctry',
       'lst_port', 'lst_city', 'lst_ctry', 'brd_port', 'brd_city', 'brd_ctry',
       'off_port', 'off_city', 'off_ctry', 'mkt_port', 'mkt_city', 'mkt_ctry',
       'intl', 'route          ', 'carrier', 'bkg_class', 'cab_class',
       'brd_time           ', 'off_time           ', 'pax', 'year', 'month',
       'oid      '],
      dtype='object')

Remove white spaces

In [22]:
columns = list(bookings_sample.columns.values)
columns_new = [column.strip() for column in columns]
bookings_sample.columns = columns_new
bookings_sample.columns 

Index(['act_date', 'source', 'pos_ctry', 'pos_iata', 'pos_oid', 'rloc',
       'cre_date', 'duration', 'distance', 'dep_port', 'dep_city', 'dep_ctry',
       'arr_port', 'arr_city', 'arr_ctry', 'lst_port', 'lst_city', 'lst_ctry',
       'brd_port', 'brd_city', 'brd_ctry', 'off_port', 'off_city', 'off_ctry',
       'mkt_port', 'mkt_city', 'mkt_ctry', 'intl', 'route', 'carrier',
       'bkg_class', 'cab_class', 'brd_time', 'off_time', 'pax', 'year',
       'month', 'oid'],
      dtype='object')

In [19]:
bookings_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 38 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   act_date             9999 non-null   object
 1   source               9999 non-null   object
 2   pos_ctry             9999 non-null   object
 3   pos_iata             9999 non-null   object
 4   pos_oid              9999 non-null   object
 5   rloc                 9999 non-null   object
 6   cre_date             9999 non-null   object
 7   duration             9999 non-null   int64 
 8   distance             9999 non-null   int64 
 9   dep_port             9999 non-null   object
 10  dep_city             9999 non-null   object
 11  dep_ctry             9999 non-null   object
 12  arr_port             9999 non-null   object
 13  arr_city             9999 non-null   object
 14  arr_ctry             9999 non-null   object
 15  lst_port             9999 non-null   object
 16  lst_ci

In [25]:
bookings_sample[['arr_port', 'pax']]
bookings_sample['pax'] = bookings_sample['pax'].dropna()


In [31]:
bookings_sample[['arr_port', 'pax']].groupby(['arr_port']).sum().sort_values('pax', ascending = False).head(10)

,pax
arr_port,
HKG,112
LGA,95
ORD,94
JFK,92
LAX,91
SFO,91
MCO,90
DCA,82
DEN,79


In [33]:
bookings_sample.groupby('arr_city').agg({"pax": "sum"}).sort_values('pax', ascending = False).head(10)

,pax
arr_city,
NYC,233
WAS,186
HKG,112
CHI,102
LON,96
LAX,91
SFO,91
ORL,90
DEN,79


In [51]:
chunk = 100000
i = 0
for x in pd.read_csv('bookings.sample.csv.bz2', sep = '^', chunksize = chunk):
    i += len(x)
print(i)

9999


In [62]:
#empty data frame
empty = pd.DataFrame()


reader = pd.read_csv('/home/dsc/Data/challenge/bookings.csv.bz2', sep = '^', chunksize = 100000)

for j, chunk in enumerate(reader):
    pax = chunk[['arr_port', 'pax']].copy()
    pax['pax'] = pax['pax'].dropna()
    pax_new = pax[['arr_port', 'pax']].groupby(['arr_port']).sum().reset_index()
    empty = empty.append(pax_new, ignore_index = True) 
    
    #stop
    if j == 3:
        break
        


In [63]:
top10 = empty[['arr_port', 'pax']].groupby(['arr_port']).sum().sort_values('pax', ascending = False).head(10)
top10

,pax
arr_port,
LHR,3950
MCO,3213
LAX,2969
LAS,2854
JFK,2804
CDG,2795
MIA,2668
SFO,2547
BKK,2539
